# F1 Prize Picks MLflow Experiment Tracking

This notebook sets up MLflow tracking for the F1 Prize Picks optimization pipeline to manage experiments, track metrics, and version models.

In [ ]:
import mlflow
import mlflow.sklearn
import mlflow.pyfunc
from mlflow.tracking import MlflowClient
from mlflow.models.signature import infer_signature
import pandas as pd
import numpy as np
import joblib
from pathlib import Path
import json
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Set up MLflow tracking URI
MLFLOW_DIR = Path('/app/notebooks/advanced/mlflow')
MLFLOW_DIR.mkdir(exist_ok=True)
mlflow.set_tracking_uri(f'file://{MLFLOW_DIR.absolute()}')

print(f"MLflow tracking URI: {mlflow.get_tracking_uri()}")

## 1. Experiment Setup and Configuration

In [ ]:
class F1MLflowTracker:
    """Manages MLflow experiment tracking for F1 predictions"""
    
    def __init__(self, experiment_name="f1_prize_picks"):
        self.experiment_name = experiment_name
        self.client = MlflowClient()
        self._setup_experiment()
        
    def _setup_experiment(self):
        """Create or get experiment"""
        try:
            self.experiment = self.client.get_experiment_by_name(self.experiment_name)
            if self.experiment is None:
                self.experiment_id = self.client.create_experiment(
                    self.experiment_name,
                    artifact_location=str(MLFLOW_DIR / "artifacts"),
                    tags={
                        "project": "F1 Prize Picks",
                        "team": "Data Science",
                        "version": "1.0"
                    }
                )
            else:
                self.experiment_id = self.experiment.experiment_id
        except Exception as e:
            print(f"Error setting up experiment: {e}")
            self.experiment_id = "0"
    
    def start_run(self, run_name=None, tags=None):
        """Start a new MLflow run"""
        if run_name is None:
            run_name = f"f1_run_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
        
        mlflow.set_experiment(self.experiment_name)
        mlflow.start_run(run_name=run_name)
        
        if tags:
            mlflow.set_tags(tags)
            
        return mlflow.active_run().info.run_id
    
    def log_model_training(self, model, X_train, y_train, X_val, y_val, 
                          model_type="random_forest", feature_names=None):
        """Log model training metrics and artifacts"""
        # Log parameters
        params = model.get_params()
        mlflow.log_params(params)
        
        # Log dataset info
        mlflow.log_param("train_samples", len(X_train))
        mlflow.log_param("val_samples", len(X_val))
        mlflow.log_param("n_features", X_train.shape[1])
        
        # Calculate and log metrics
        from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
        
        train_pred = model.predict(X_train)
        val_pred = model.predict(X_val)
        
        metrics = {
            "train_accuracy": accuracy_score(y_train, train_pred),
            "val_accuracy": accuracy_score(y_val, val_pred),
            "train_precision": precision_score(y_train, train_pred, average='weighted'),
            "val_precision": precision_score(y_val, val_pred, average='weighted'),
            "train_recall": recall_score(y_train, train_pred, average='weighted'),
            "val_recall": recall_score(y_val, val_pred, average='weighted'),
            "train_f1": f1_score(y_train, train_pred, average='weighted'),
            "val_f1": f1_score(y_val, val_pred, average='weighted')
        }
        
        mlflow.log_metrics(metrics)
        
        # Log feature importance if available
        if hasattr(model, 'feature_importances_') and feature_names:
            importance_df = pd.DataFrame({
                'feature': feature_names,
                'importance': model.feature_importances_
            }).sort_values('importance', ascending=False)
            
            # Save as artifact
            importance_path = "/tmp/feature_importance.csv"
            importance_df.to_csv(importance_path, index=False)
            mlflow.log_artifact(importance_path)
        
        # Log model
        signature = infer_signature(X_train, train_pred)
        mlflow.sklearn.log_model(
            model, 
            f"{model_type}_model",
            signature=signature
        )
        
        return metrics
    
    def log_prediction_results(self, predictions_df, actual_results=None):
        """Log prediction results and performance"""
        # Log prediction statistics
        mlflow.log_metric("n_predictions", len(predictions_df))
        
        if 'probability' in predictions_df.columns:
            mlflow.log_metric("avg_confidence", predictions_df['probability'].mean())
            mlflow.log_metric("std_confidence", predictions_df['probability'].std())
        
        # If we have actual results, calculate performance
        if actual_results is not None:
            merged = predictions_df.merge(actual_results, on=['race_id', 'driver_id'])
            accuracy = (merged['predicted'] == merged['actual']).mean()
            mlflow.log_metric("prediction_accuracy", accuracy)
        
        # Save predictions as artifact
        predictions_path = "/tmp/predictions.csv"
        predictions_df.to_csv(predictions_path, index=False)
        mlflow.log_artifact(predictions_path)
    
    def log_betting_performance(self, bets_df, bankroll_history):
        """Log betting performance metrics"""
        # Calculate key metrics
        total_bets = len(bets_df)
        winning_bets = (bets_df['payout'] > 0).sum()
        total_wagered = bets_df['stake'].sum()
        total_payout = bets_df['payout'].sum()
        roi = (total_payout - total_wagered) / total_wagered if total_wagered > 0 else 0
        
        mlflow.log_metrics({
            "total_bets": total_bets,
            "winning_bets": winning_bets,
            "win_rate": winning_bets / total_bets if total_bets > 0 else 0,
            "total_wagered": total_wagered,
            "total_payout": total_payout,
            "roi": roi,
            "final_bankroll": bankroll_history[-1] if bankroll_history else 0
        })
        
        # Log Sharpe ratio if we have enough data
        if len(bankroll_history) > 10:
            returns = pd.Series(bankroll_history).pct_change().dropna()
            sharpe = returns.mean() / returns.std() * np.sqrt(252) if returns.std() > 0 else 0
            mlflow.log_metric("sharpe_ratio", sharpe)
        
        # Save betting history
        bets_path = "/tmp/betting_history.csv"
        bets_df.to_csv(bets_path, index=False)
        mlflow.log_artifact(bets_path)
        
        # Save bankroll evolution
        bankroll_df = pd.DataFrame({
            'period': range(len(bankroll_history)),
            'bankroll': bankroll_history
        })
        bankroll_path = "/tmp/bankroll_history.csv"
        bankroll_df.to_csv(bankroll_path, index=False)
        mlflow.log_artifact(bankroll_path)
    
    def end_run(self):
        """End the current MLflow run"""
        mlflow.end_run()

# Initialize tracker
tracker = F1MLflowTracker()
print(f"Experiment ID: {tracker.experiment_id}")

## 2. Custom Model Wrapper for Prize Picks

In [ ]:
class PrizePicksMLflowModel(mlflow.pyfunc.PythonModel):
    """Custom MLflow model for Prize Picks predictions"""
    
    def __init__(self, models_dict, feature_columns, optimizer_config):
        self.models_dict = models_dict
        self.feature_columns = feature_columns
        self.optimizer_config = optimizer_config
    
    def predict(self, context, model_input):
        """Generate Prize Picks recommendations"""
        predictions = []
        
        for prop_type, model in self.models_dict.items():
            if prop_type in model_input.columns:
                # Get features for this prop type
                features = model_input[self.feature_columns[prop_type]]
                
                # Make predictions
                probs = model.predict_proba(features)[:, 1]
                
                # Apply optimizer logic
                threshold = self.optimizer_config.get('threshold', 0.55)
                high_confidence = probs > threshold
                
                prop_predictions = pd.DataFrame({
                    'prop_type': prop_type,
                    'probability': probs,
                    'recommended': high_confidence,
                    'kelly_fraction': self._calculate_kelly(probs)
                })
                
                predictions.append(prop_predictions)
        
        return pd.concat(predictions, ignore_index=True)
    
    def _calculate_kelly(self, probs):
        """Calculate Kelly fraction for bet sizing"""
        # Simplified Kelly calculation
        odds = 2.0  # Assume even money for Prize Picks
        edge = probs * odds - 1
        kelly = np.where(edge > 0, edge / (odds - 1), 0)
        return np.clip(kelly * 0.25, 0, 0.1)  # Conservative Kelly

## 3. Experiment Tracking Integration

In [ ]:
def track_full_pipeline_run(pipeline_config, results):
    """Track a complete pipeline run in MLflow"""
    
    # Start run
    run_id = tracker.start_run(
        run_name=f"pipeline_{pipeline_config.get('race_id', 'test')}",
        tags={
            "pipeline_version": pipeline_config.get('version', '1.0'),
            "environment": pipeline_config.get('environment', 'development'),
            "race_id": str(pipeline_config.get('race_id', 'N/A'))
        }
    )
    
    try:
        # Log configuration
        mlflow.log_params({
            "optimizer_strategy": pipeline_config.get('optimizer_strategy', 'conservative'),
            "kelly_fraction": pipeline_config.get('kelly_fraction', 0.25),
            "min_edge": pipeline_config.get('min_edge', 0.05),
            "max_picks_per_parlay": pipeline_config.get('max_picks_per_parlay', 6),
            "bankroll": pipeline_config.get('bankroll', 100)
        })
        
        # Log results
        if 'predictions' in results:
            tracker.log_prediction_results(results['predictions'])
        
        if 'betting_results' in results:
            tracker.log_betting_performance(
                results['betting_results'],
                results.get('bankroll_history', [])
            )
        
        # Log performance metrics
        if 'metrics' in results:
            mlflow.log_metrics(results['metrics'])
        
        # Save pipeline artifacts
        if 'report' in results:
            report_path = "/tmp/pipeline_report.json"
            with open(report_path, 'w') as f:
                json.dump(results['report'], f, indent=2)
            mlflow.log_artifact(report_path)
        
        print(f"Run logged successfully: {run_id}")
        
    except Exception as e:
        print(f"Error logging run: {e}")
        mlflow.set_tag("error", str(e))
    finally:
        tracker.end_run()
    
    return run_id

# Example usage
example_config = {
    'race_id': 'test_race_2024',
    'version': '1.0',
    'optimizer_strategy': 'moderate',
    'kelly_fraction': 0.25,
    'bankroll': 1000
}

example_results = {
    'predictions': pd.DataFrame({
        'driver_id': [1, 2, 3],
        'prop_type': ['winner', 'podium', 'points'],
        'probability': [0.65, 0.72, 0.58]
    }),
    'metrics': {
        'avg_confidence': 0.65,
        'n_high_value_picks': 5
    }
}

# Track example run
# run_id = track_full_pipeline_run(example_config, example_results)

## 4. Model Registry and Versioning

In [ ]:
class F1ModelRegistry:
    """Manage model versions in MLflow"""
    
    def __init__(self):
        self.client = MlflowClient()
    
    def register_model(self, run_id, model_name, model_type="winner_prediction"):
        """Register a model from a run"""
        model_uri = f"runs:/{run_id}/{model_type}_model"
        
        try:
            # Register the model
            mv = mlflow.register_model(model_uri, model_name)
            
            # Add description and tags
            self.client.update_model_version(
                name=model_name,
                version=mv.version,
                description=f"F1 {model_type} model trained on {datetime.now().date()}"
            )
            
            self.client.set_model_version_tag(
                name=model_name,
                version=mv.version,
                key="model_type",
                value=model_type
            )
            
            print(f"Model registered: {model_name} v{mv.version}")
            return mv
            
        except Exception as e:
            print(f"Error registering model: {e}")
            return None
    
    def promote_model(self, model_name, version, stage="Production"):
        """Promote a model version to a stage"""
        try:
            self.client.transition_model_version_stage(
                name=model_name,
                version=version,
                stage=stage,
                archive_existing_versions=True
            )
            print(f"Model {model_name} v{version} promoted to {stage}")
        except Exception as e:
            print(f"Error promoting model: {e}")
    
    def get_latest_model(self, model_name, stage="Production"):
        """Get the latest model version for a stage"""
        try:
            versions = self.client.get_latest_versions(model_name, stages=[stage])
            if versions:
                return versions[0]
            else:
                print(f"No {stage} version found for {model_name}")
                return None
        except Exception as e:
            print(f"Error getting model: {e}")
            return None
    
    def load_production_model(self, model_name):
        """Load the production version of a model"""
        model_version = self.get_latest_model(model_name, "Production")
        if model_version:
            model_uri = f"models:/{model_name}/{model_version.version}"
            return mlflow.sklearn.load_model(model_uri)
        return None

# Initialize registry
registry = F1ModelRegistry()

## 5. Experiment Comparison and Analysis

In [ ]:
def compare_experiments(experiment_name="f1_prize_picks", metric="val_accuracy"):
    """Compare runs within an experiment"""
    
    # Get experiment
    experiment = mlflow.get_experiment_by_name(experiment_name)
    if not experiment:
        print(f"Experiment {experiment_name} not found")
        return None
    
    # Search runs
    runs = mlflow.search_runs(
        experiment_ids=[experiment.experiment_id],
        order_by=[f"metrics.{metric} DESC"],
        max_results=20
    )
    
    if runs.empty:
        print("No runs found")
        return None
    
    # Display key metrics
    display_cols = [
        'run_id', 'start_time', f'metrics.{metric}',
        'metrics.val_f1', 'metrics.roi', 'metrics.sharpe_ratio',
        'params.optimizer_strategy', 'params.kelly_fraction'
    ]
    
    # Filter existing columns
    display_cols = [col for col in display_cols if col in runs.columns]
    
    comparison_df = runs[display_cols].head(10)
    
    # Plot performance
    if f'metrics.{metric}' in runs.columns:
        import matplotlib.pyplot as plt
        
        plt.figure(figsize=(10, 6))
        plt.plot(runs['start_time'], runs[f'metrics.{metric}'], 'o-')
        plt.xlabel('Run Time')
        plt.ylabel(metric)
        plt.title(f'{metric} Over Time')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()
    
    return comparison_df

# Compare experiments
# comparison = compare_experiments()

## 6. Integration with Pipeline

In [ ]:
def create_mlflow_callback():
    """Create callback for pipeline integration"""
    
    def mlflow_callback(stage, data):
        """Callback to log pipeline stages to MLflow"""
        
        if stage == "data_loaded":
            mlflow.log_metric("n_races", len(data.get('races', [])))
            mlflow.log_metric("n_drivers", len(data.get('drivers', [])))
            
        elif stage == "features_computed":
            features_df = data.get('features')
            if features_df is not None:
                mlflow.log_metric("n_features", features_df.shape[1])
                mlflow.log_metric("n_samples", features_df.shape[0])
                
        elif stage == "predictions_made":
            predictions = data.get('predictions')
            if predictions is not None:
                mlflow.log_metric("n_predictions", len(predictions))
                if 'probability' in predictions.columns:
                    mlflow.log_metric("avg_confidence", predictions['probability'].mean())
                    
        elif stage == "optimization_complete":
            picks = data.get('optimized_picks')
            if picks:
                mlflow.log_metric("n_recommended_picks", len(picks))
                mlflow.log_metric("total_expected_value", 
                                sum(p.get('expected_value', 0) for p in picks))
                
        elif stage == "pipeline_complete":
            # Log final artifacts
            if 'report_path' in data:
                mlflow.log_artifact(data['report_path'])
            
            # Log summary metrics
            summary = data.get('summary', {})
            for key, value in summary.items():
                if isinstance(value, (int, float)):
                    mlflow.log_metric(f"summary_{key}", value)
    
    return mlflow_callback

# Example: Integrate with pipeline
print("MLflow callback created for pipeline integration")
print("Use: pipeline.add_callback(create_mlflow_callback())")

## 7. MLflow UI Launch Helper

In [ ]:
# Create launch script for MLflow UI
launch_script = """#!/bin/bash
# Launch MLflow UI for F1 Prize Picks experiments

echo "Starting MLflow UI..."
echo "Access at: http://localhost:5000"
echo "Press Ctrl+C to stop"

cd /app/notebooks/advanced
mlflow ui --backend-store-uri file:///app/notebooks/advanced/mlflow --port 5000
"""

with open('/app/notebooks/advanced/mlflow/launch_ui.sh', 'w') as f:
    f.write(launch_script)

import os
os.chmod('/app/notebooks/advanced/mlflow/launch_ui.sh', 0o755)

print("MLflow UI launch script created at:")
print("/app/notebooks/advanced/mlflow/launch_ui.sh")
print("\nTo start MLflow UI, run:")
print("bash /app/notebooks/advanced/mlflow/launch_ui.sh")

## Summary

This notebook has set up comprehensive MLflow tracking for the F1 Prize Picks pipeline:

1. **Experiment Tracking**: Automated logging of model training, predictions, and betting performance
2. **Model Registry**: Version control and lifecycle management for models
3. **Custom Model Wrapper**: Prize Picks specific model implementation
4. **Pipeline Integration**: Callbacks for seamless tracking during pipeline execution
5. **Experiment Comparison**: Tools to analyze and compare different runs
6. **MLflow UI**: Launch script for visual experiment tracking

### Usage in Pipeline:

```python
# In your pipeline code
from F1_MLflow_Tracking import F1MLflowTracker, create_mlflow_callback

# Initialize tracker
tracker = F1MLflowTracker()

# Add callback to pipeline
pipeline.add_callback(create_mlflow_callback())

# Track run
run_id = tracker.start_run()
# ... run pipeline ...
tracker.end_run()
```

### Next Steps:
1. Integrate MLflow tracking into the main pipeline
2. Set up automated model promotion based on performance
3. Create dashboards for monitoring betting performance
4. Implement A/B testing for different strategies